In [ ]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.path.join(os.getcwd(), ".."), ".."))
sys.path.append(parent_dir)
from pathlib import Path

import numpy as np
from data_generation.models.tech_substitution import TechnologySubstitution, TechSubNumericalSolver
from data_generation.models.general_ode_solver import FitzHughNagumoModel
from data_generation.models.general_ode_solver import GeneralODENumericalSolver
from data_generation.simulations.grid import Grid, fractional_transformation, logistic_transformation
from data_generation.simulations.simulator import Simulator

In [2]:
OUTPUT_DIR = Path.cwd().parent.parent / "datasets" / "results"
OUTPUT_DIR

PosixPath('/home/kstiller/masterthesis/Coding/mdp-world-model/datasets/results')

In [3]:
def test_simulation_storage(output_dir, bounds, transformations, model, solver, control):
    """Test simulating and storing simulation results in a SQLite database.
    
    Args:
        output_dir (str or Path): Absolute path to output directory
        bounds (list): List of tuples containing the lower and upper bounds of the grid
        transformations (list): List of transformation functions
        model : Model to simulate
        solver(model): Solver to use, already initialized with the model
        control (scalar or array): Control parameters for the simulation
    """
    # Setup paths
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Setup simulation
    resolution = [7, 7]
       
    # Create objects
    grid = Grid(bounds, resolution, transformations)
    simulator = Simulator(grid, model, solver)
    
    # Run simulations
    results = simulator.simulate(
        control=control,
        delta_t=0.1,
        num_samples_per_cell=3,  
        num_steps=50,
        save_result=True
    )
    
    # Store results
    db_path = output_path / 'simulation_results_test.db'
    simulator.store_results_to_sqlite(db_path)
    
    print(f"Stored {len(results)} rows of simulation data in {db_path}")

    # Print info about stored data
    print("\nConfigs data:")
    print(simulator.configs.head())
    print("\nResults data:")
    print(simulator.results.head())
    
    return simulator

In [ ]:
bounds = [(0, np.inf), (0, np.inf)]
transformations = [fractional_transformation(3), fractional_transformation(3)]
model = TechnologySubstitution()
num_solver = TechSubNumericalSolver(model)
control = 0.5

In [5]:
simulator = test_simulation_storage(OUTPUT_DIR, bounds, transformations, model, num_solver, control)

Simulation complete:
- 147 samples × 50 timesteps = 7350 total rows
- State dimensions: 2
- Control dimensions: 1
Saved results and config.
Stored 7350 rows of simulation data in /home/kstiller/masterthesis/Coding/mdp-world-model/datasets/results/simulation_results_test.db

Configs data:
            run_id                                           metadata
0  20250210_113054  {"configurations": {"grid": {"bounds": [[0, In...

Results data:
            run_id trajectory_id   t0   t1        x0        x1   c0        y0  \
0  20250210_113054         0-0_0  0.0  0.1  0.454042  0.255848  0.5  0.456911   
1  20250210_113054         0-0_0  0.1  0.2  0.456911  0.280157  0.5  0.459451   
2  20250210_113054         0-0_0  0.2  0.3  0.459451  0.306763  0.5  0.461681   
3  20250210_113054         0-0_0  0.3  0.4  0.461681  0.335679  0.5  0.463632   
4  20250210_113054         0-0_0  0.4  0.5  0.463632  0.366861  0.5  0.465331   

         y1  
0  0.280157  
1  0.306763  
2  0.335679  
3  0.366861  

In [ ]:
bounds = [(-np.inf, np.inf), (-np.inf, np.inf)]
trafo = logistic_transformation({'k': 1, 'x_0': 0})
transformations = [trafo, trafo]
model = FitzHughNagumoModel(control_params=['I', 'a'])
num_solver = GeneralODENumericalSolver(model)
control = [0.7, 0.5]

In [9]:
simulator = test_simulation_storage(OUTPUT_DIR, bounds, transformations, model, num_solver, control)

Simulation complete:
- 147 samples × 50 timesteps = 7350 total rows
- State dimensions: 2
- Control dimensions: 2
Saved results and config.
Stored 7350 rows of simulation data in /home/kstiller/masterthesis/Coding/mdp-world-model/datasets/results/simulation_results_test.db

Configs data:
            run_id                                           metadata
0  20250210_113147  {"configurations": {"grid": {"bounds": [[-Infi...

Results data:
            run_id trajectory_id   t0   t1        x0        x1   c0   c1  \
0  20250210_113147         0-0_0  0.0  0.1 -2.437791 -3.375719  0.7  0.5   
1  20250210_113147         0-0_0  0.1  0.2 -1.972336 -2.546094  0.7  0.5   
2  20250210_113147         0-0_0  0.2  0.3 -1.629230 -2.084501  0.7  0.5   
3  20250210_113147         0-0_0  0.3  0.4 -1.341994 -1.759565  0.7  0.5   
4  20250210_113147         0-0_0  0.4  0.5 -1.080005 -1.498683  0.7  0.5   

         y0        y1  
0 -1.972336 -2.546094  
1 -1.629230 -2.084501  
2 -1.341994 -1.759565  
3 -